In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [21]:
warnings.simplefilter(action='ignore', category=FutureWarning)

# Sopprimi i RuntimeWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [22]:
dati = pd.read_csv('Bitcoin (€) da yfinance dal 17-09-2014 al 24-04-2024.csv')
dati = dati[['Date', 'Close']]
dati = dati.rename(columns = {'Date':'Timestamp', 'Close': 'Price'})
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'])
dati['Timestamp'] = dati['Timestamp'].dt.strftime('%Y-%m-%d')
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'], format='%Y-%m-%d')
dati

,Timestamp,Price
0,2014-09-17,355.957367
1,2014-09-18,328.539368
2,2014-09-19,307.761139
3,2014-09-20,318.758972
4,2014-09-21,310.632446
...,...,...
3503,2024-04-20,59876.710938
3504,2024-04-21,60956.074219
3505,2024-04-22,60919.242188
3506,2024-04-23,62729.296875


In [23]:
data_inizio = '2014-09-17'
data_fine = '2024-04-24'
indice_inizio = dati[dati['Timestamp'] == data_inizio].index[0]
indice_fine = dati[dati['Timestamp'] == data_fine].index[0]
dati = dati[indice_inizio:indice_fine + 1].reset_index(drop=True)
dati

,Timestamp,Price
0,2014-09-17,355.957367
1,2014-09-18,328.539368
2,2014-09-19,307.761139
3,2014-09-20,318.758972
4,2014-09-21,310.632446
...,...,...
3503,2024-04-20,59876.710938
3504,2024-04-21,60956.074219
3505,2024-04-22,60919.242188
3506,2024-04-23,62729.296875


In [24]:
def calcola_rendimento_portafoglio_ideale(lista_prezzi, capitale_iniziale, bitcoin_iniziali, min_acquisto=0, min_vendita=0, perc_commissione_acquisto=0, perc_commissione_vendita=0):
    capitale = capitale_iniziale
    bitcoin = bitcoin_iniziali
    comprato = False

    for i in range(len(lista_prezzi) - 1):
        # Compra al minimo
        if lista_prezzi[i] < lista_prezzi[i + 1] and not comprato and capitale >= min_acquisto:
            bitcoin_acquistati = (capitale * (1 - perc_commissione_acquisto)) / lista_prezzi[i]
            bitcoin += bitcoin_acquistati
            capitale = 0
            comprato = True
            # print(f'Bitcoin posseduti: {bitcoin}')
        # Vende al massimo
        elif lista_prezzi[i] > lista_prezzi[i + 1] and comprato and bitcoin * lista_prezzi[i] >= min_vendita:
            capitale_vendita = bitcoin * lista_prezzi[i] * (1 - perc_commissione_vendita)
            capitale += capitale_vendita
            bitcoin = 0
            comprato = False
            # print(f'Capitale posseduto: {capitale}')

    # Vende i bitcoin rimasti all'ultimo prezzo
    if comprato and bitcoin * lista_prezzi.iloc[-1] >= min_vendita:
        capitale += bitcoin * lista_prezzi.iloc[-1] * (1 - perc_commissione_vendita)
        bitcoin = 0
        # print(f'Capitale posseduto: {capitale}')
        # print(f'Bitcoin posseduti: {bitcoin}')

    valore_iniziale = capitale_iniziale + bitcoin_iniziali * lista_prezzi[0]
    valore_finale = capitale + bitcoin * lista_prezzi.iloc[-1]

    rendimento = valore_finale / valore_iniziale

    return [rendimento, valore_finale]

In [30]:
def estrai_periodi_casuali(dati, N, L, start_date, end_date, random_state):
    # Filtra i dati tra start_date e end_date
    dati_filtrati = dati[(dati['Timestamp'] >= pd.to_datetime(start_date)) & 
                         (dati['Timestamp'] <= pd.to_datetime(end_date))]
    
    # Calcola il numero massimo di periodi non sovrapposti
    num_max_periodi = len(dati_filtrati) - L + 1
    
    # Controlla se è possibile estrarre N periodi diversi
    if num_max_periodi < N:
        raise ValueError(f"Non è possibile estrarre {N} periodi diversi di lunghezza {L} tra le date {start_date} e {end_date}. Possono essere al massimo {num_max_periodi}.")
    
    # Inizializza una lista per i periodi estratti e un set per memorizzare gli estremi
    periodi_estratti = []
    estremi_estratti = set()
    
    # Inizializza il generatore di numeri casuali con il seed dato
    rng = np.random.RandomState(random_state)
    
    while len(periodi_estratti) < N:
        # Seleziona un indice casuale per l'inizio del periodo
        indice_inizio = rng.randint(0, len(dati_filtrati) - L + 1)
        
        # Determina gli estremi del periodo
        estremo_inizio = dati_filtrati.iloc[indice_inizio]['Timestamp']
        estremo_fine = dati_filtrati.iloc[indice_inizio + L - 1]['Timestamp']
        
        # Controlla se gli estremi sono già stati utilizzati
        if (estremo_inizio, estremo_fine) not in estremi_estratti:
            # Aggiungi gli estremi al set
            estremi_estratti.add((estremo_inizio, estremo_fine))
            
            # Estrai il periodo e aggiungilo alla lista dei periodi estratti
            periodo = dati_filtrati.iloc[indice_inizio:indice_inizio + L]

            periodo = periodo.reset_index(drop=True)

            periodi_estratti.append(periodo)
    
    return periodi_estratti

In [35]:
# Esempio di utilizzo:
start_date = dati['Timestamp'].iloc[0]
end_date = dati['Timestamp'].iloc[-1]
N = 3000 # Numero di periodi da estrarre
L = 60 # Lunghezza di ogni periodo in giorni
random_state = 8

In [36]:
periodi_casuali_esistenti = estrai_periodi_casuali(dati, N, L, start_date, end_date, random_state)
print('Fattore di ritorno medio dei prezzi estratti:', np.mean([df['Price'].iloc[-1] / df['Price'].iloc[0] for df in periodi_casuali_esistenti]))

ideal_returns = [calcola_rendimento_portafoglio_ideale(df['Price'], 1000, 0, 5, 5, 0.001, 0.001)[0] for df in periodi_casuali_esistenti]

print('Media dei fattori di rendimento massimi ottenibili:', np.mean(ideal_returns))

perc_suff = np.mean([1/calcola_rendimento_portafoglio_ideale(df['Price'], 1000, 0, 5, 5, 0.001, 0.001)[0] for df in periodi_casuali_esistenti])

print(f'Punteggio percentuale considerato "sufficiente": {round(perc_suff*100,2)}%')

Fattore di ritorno medio dei prezzi estratti: 1.1500421594461692
Media dei fattori di rendimento massimi ottenibili: 2.2005397172421484
Punteggio percentuale considerato "sufficiente": 51.01%


In [37]:
# datasets = [periodi_casuali_esistenti[i]['Price'].values for i in range(len(periodi_casuali_esistenti))]

# lunghezza_dataset = len(periodi_casuali_esistenti[0])

# # Converte i dataset generati in DataFrame
# generated_dfs = [pd.DataFrame({'Timestamp': dati['Timestamp'].iloc[:lunghezza_dataset], 'Price': prices}) for prices in datasets] # Fingo che le date siano tutte iniziate con data_inizio

# # Plotting
# plt.figure(figsize=(14, 7))

# # Plot del dataset originale
# plt.plot(dati['Timestamp'].iloc[:lunghezza_dataset], dati['Price'].iloc[:lunghezza_dataset].values, label='Original', linewidth=3)

# # Plot dei dataset generati
# for i, df in enumerate(generated_dfs):
#     plt.plot(df['Timestamp'], df['Price'], label=f'Generated Dataset {i+1}', alpha=0.7)

# plt.title('Generated Bitcoin Price Datasets')
# plt.xlabel('Time')
# plt.ylabel('Price')
# # plt.yscale('log')
# plt.grid()
# # plt.legend()
# plt.show()

In [38]:
dati_da_usare = periodi_casuali_esistenti.copy()

In [40]:
def calcola_punteggio_bullish(dataset):
    variazioni = np.diff(dataset['Price'])
    salite = np.sum(variazioni[variazioni > 0])
    discese = np.sum(np.abs(variazioni[variazioni < 0]))
    punteggio = 2 * (salite / (salite + discese)) - 1
    return punteggio

punteggi_normalizzati = [calcola_punteggio_bullish(df) for df in dati_da_usare]

In [41]:
punteggi_normalizzati[:10]

[np.float64(-0.126712774985569),
 np.float64(-0.020462544807182237),
 np.float64(0.20349283878871316),
 np.float64(-0.1561117865673638),
 np.float64(0.42666530557213167),
 np.float64(0.08906025558049446),
 np.float64(0.40645977837425407),
 np.float64(-0.09823852535198097),
 np.float64(0.03823084154498968),
 np.float64(0.18878346211682095)]

In [42]:
# Dividi i punteggi normalizzati in n_clusters intervalli
n_clusters = 7
intervallo = np.linspace(-1, 1, n_clusters + 1)
clusters = np.digitize(punteggi_normalizzati, intervallo)

# Calcola le statistiche per ogni cluster
statistiche = []
for i in range(n_clusters):
    # Utilizza una lista di booleani per selezionare i dataset appartenenti al cluster i
    cluster_i = [df for df, cluster in zip(dati_da_usare, clusters) if cluster == i]
    punteggi_cluster_i = [punteggio for punteggio, cluster in zip(punteggi_normalizzati, clusters) if cluster == i]

    statistiche.append({
        'Cluster': i+1,
        'Intervallo': intervallo[i:i+2],
        'Numero di dataset': len(cluster_i),
        'Media del punteggio': np.mean(punteggi_cluster_i) if punteggi_cluster_i else 0,
        'Mediana del punteggio': np.median(punteggi_cluster_i) if punteggi_cluster_i else 0,
        'Deviazione standard del punteggio': np.std(punteggi_cluster_i) if punteggi_cluster_i else 0,
        'Minimo del punteggio': np.min(punteggi_cluster_i) if punteggi_cluster_i else 0,
        'Massimo del punteggio': np.max(punteggi_cluster_i) if punteggi_cluster_i else 0
    })

# Visualizza le statistiche
df_statistiche = pd.DataFrame(statistiche)
df_statistiche

,Cluster,Intervallo,Numero di dataset,Media del punteggio,Mediana del punteggio,Deviazione standard del punteggio,Minimo del punteggio,Massimo del punteggio
0,1,"[-1.0, -0.7142857142857143]",0,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,"[-0.7142857142857143, -0.4285714285714286]",0,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,"[-0.4285714285714286, -0.1428571428571429]",9,-0.476993,-0.485191,0.023696,-0.516513,-0.443653
3,4,"[-0.1428571428571429, 0.1428571428571428]",544,-0.223674,-0.212272,0.060414,-0.422608,-0.143197
4,5,"[0.1428571428571428, 0.4285714285714284]",1316,0.003172,0.003321,0.083468,-0.142730,0.142637
5,6,"[0.4285714285714284, 0.7142857142857142]",989,0.259807,0.245151,0.082126,0.142903,0.428195
6,7,"[0.7142857142857142, 1.0]",142,0.490154,0.473648,0.052693,0.429715,0.642298
